# QC `ntd_id_rtpa_crosswalk` against `dim_organizations`

As of 4/21/2025, `dim_orgs` now has an RTPA column. RTPA values were set from a previous `rtpa/mpo` column. The `ntd_id_rtpa_crosswalk`.
was created by pulling all the agnecies in ntd and assigning a rtpa based on their city location.

## Tasks
- see if the ntd_id in the xwalk appear in dim_orgs and vise-versa.
    - are all ntd_id from xwalk in dim_orgs? NO
    - are all ntd_id from dim_orgs in xwalk? NO!
    
- do the ntd_id/rtpa pairs from xwalk match to dim_orgs
    - SCAG acounts for like 6 counties in Socal

- refactor analyses that use the old xwalk
    - remove old xwalk
    - replace with rtpa data from dim_orgs
    - make manual adjustments to SCAG agengies >> separate out to the socal CTCs (orange, imperial, san bernardino etc etc)


---
Findings
- there are some ntd_id that are unique to dim_orgs that are not in my xwalk
- vise-versa, there are some ntd_id that are unique to xwalk but are not in dim_orgs
- `Agency Status`: Designates agencies as active or inactive based on whether or not they submitted a report in the most recent report year.					


work-plan
- fill in the missing RTPA columns in dim_orgs
- what is the process of adding new orgs to dim_orgs? sounds like a lot of work (filling in all the columns)

In [1]:
import pandas as pd
from calitp_data_analysis.tables import tbls
from siuba import _, collect, count, distinct, filter, select, show_query

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [2]:
xwalk = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk_all_reporter_types.parquet"
)

In [3]:
dim_orgs = (
    tbls.mart_transit_database.dim_organizations()
    >> filter(
        _._is_current == True,
        # _.public_currently_operating == True,
        _.ntd_id_2022 != "",
    )
    >> select(
        _.name,
        _.ntd_id_2022,
        _.rtpa_name,
        _.key,
        # _.alias # alias is a list that is messing up merges down the line
    )
    >> collect()
)

dim_orgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         239 non-null    object
 1   ntd_id_2022  239 non-null    object
 2   rtpa_name    233 non-null    object
 3   key          239 non-null    object
dtypes: object(4)
memory usage: 7.6+ KB


In [4]:
currently_operating = (
    tbls.mart_transit_database.dim_organizations()
    >> filter(
        _._is_current == True, _.public_currently_operating == True, _.ntd_id_2022 != ""
    )
    >> select(
        _.name,
        _.ntd_id_2022,
        _.rtpa_name,
        _.key,
        # _.alias
    )
    >> collect()
)

currently_operating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         183 non-null    object
 1   ntd_id_2022  183 non-null    object
 2   rtpa_name    183 non-null    object
 3   key          183 non-null    object
dtypes: object(4)
memory usage: 5.8+ KB


In [5]:
currently_operating[currently_operating["ntd_id_2022"].isna()]

,name,ntd_id_2022,rtpa_name,key


In [6]:
# need to get county info?
county_bridge = (
    tbls.mart_transit_database.bridge_organizations_x_headquarters_county_geography()
    >> filter(
        _._is_current == True,
    )
    >> select(
        _.organization_key,
        _.organization_name,
        _.county_geography_name,
        _.county_geography_key,
    )
    >> collect()
)

county_bridge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1294 entries, 0 to 1293
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   organization_key       1294 non-null   object
 1   organization_name      1294 non-null   object
 2   county_geography_name  1294 non-null   object
 3   county_geography_key   1294 non-null   object
dtypes: object(4)
memory usage: 40.6+ KB


In [7]:
dim_org_county = dim_orgs.merge(
    county_bridge, how="left", left_on="key", right_on="organization_key"
)

display(
    dim_org_county.info(),
    # dim_org_county.head()
)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 0 to 238
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   239 non-null    object
 1   ntd_id_2022            239 non-null    object
 2   rtpa_name              233 non-null    object
 3   key                    239 non-null    object
 4   organization_key       239 non-null    object
 5   organization_name      239 non-null    object
 6   county_geography_name  239 non-null    object
 7   county_geography_key   239 non-null    object
dtypes: object(8)
memory usage: 16.8+ KB


None

In [8]:
dim_org_county_2 = currently_operating.merge(
    county_bridge, how="left", left_on="key", right_on="organization_key"
)

display(
    dim_org_county_2.info(),
    # dim_org_county.head()
)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 0 to 182
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   name                   183 non-null    object
 1   ntd_id_2022            183 non-null    object
 2   rtpa_name              183 non-null    object
 3   key                    183 non-null    object
 4   organization_key       183 non-null    object
 5   organization_name      183 non-null    object
 6   county_geography_name  183 non-null    object
 7   county_geography_key   183 non-null    object
dtypes: object(8)
memory usage: 12.9+ KB


None

## What is the differences in NTD id between dim_org_county and dim_org_county_2?


In [9]:
id_check = dim_org_county.merge(dim_org_county_2, how="outer", indicator=True)

In [10]:
display(
    id_check["_merge"].value_counts(),
)

both          183
left_only      56
right_only      0
Name: _merge, dtype: int64

### Decision

It is possible for agencies to switch between currently operating and not operating. Therefore, RTPA values should be assigned to agencies regardless of operating status.

## What are the unique pairs of RTPA names and geography names?

In [11]:
rtpa_county = (
    dim_org_county[dim_org_county["rtpa_name"].notna()][
        ["county_geography_name", "rtpa_name"]
    ]
    .drop_duplicates()
    .sort_values(by="rtpa_name")
)
rtpa_county[rtpa_county["county_geography_name"].str.contains("Plumas")]

,county_geography_name,rtpa_name
55,Plumas,Plumas County Transportation Commission


## Create dictionary of counties names : RTPA name

In [12]:
county_rtpa_dict = rtpa_county.set_index("county_geography_name")["rtpa_name"].to_dict()
county_rtpa_dict.update(
    {
        "Plumas": "Plumas County Transportation Commission",
        "Sierra": "Sierra County Transportation Commission",
        # "Imperial":"Imperial County Transportation Commission", NO! Imperial is under SCAG
        # "Los Angeles":"Los Angeles County Metropolitan Transportation Authority", NO! LA is SCAG
    }
)
county_rtpa_dict["Imperial"]

'Southern California Association of Governments'

## What are the Agencies with missing RTPA names, but have a county name?

In [13]:
dim_org_county[
    (dim_org_county["rtpa_name"].isna())
    & (dim_org_county["county_geography_name"].isna())
].drop_duplicates()

# added rtpa names in airtable

,name,ntd_id_2022,rtpa_name,key,organization_key,organization_name,county_geography_name,county_geography_key


In [14]:
no_rtpa = dim_org_county[
    (dim_org_county["rtpa_name"].isna())
    & (dim_org_county["county_geography_name"].notna())
].drop_duplicates()

len(no_rtpa)

6

## !!THESE ORGS NEED RTPAs IN dim_orgs!!

In [15]:
no_rtpa["update_dim_org_rtpa_to"] = no_rtpa["county_geography_name"].map(
    county_rtpa_dict
)

In [16]:
display(
    no_rtpa.info(),
    no_rtpa[
        [
            "name",
            "ntd_id_2022",
            "rtpa_name",
            # "alias",
            "county_geography_name",
            "update_dim_org_rtpa_to",
        ]
    ].sort_values(by="county_geography_name"),
)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 4 to 68
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    6 non-null      object
 1   ntd_id_2022             6 non-null      object
 2   rtpa_name               0 non-null      object
 3   key                     6 non-null      object
 4   organization_key        6 non-null      object
 5   organization_name       6 non-null      object
 6   county_geography_name   6 non-null      object
 7   county_geography_key    6 non-null      object
 8   update_dim_org_rtpa_to  6 non-null      object
dtypes: object(9)
memory usage: 480.0+ bytes


None

,name,ntd_id_2022,rtpa_name,county_geography_name,update_dim_org_rtpa_to
63,San Gabriel Transit Inc.,90187,None,Los Angeles,Southern California Association of Governments
30,City of Merced,90143,None,Merced,Merced County Association of Governments
4,California Department of Transportation,9R02,None,Sacramento,Sacramento Area Council of Governments
62,"San Diego Trolley, Inc.",90054,None,San Diego,San Diego Association of Governments
11,City of Benicia,90174,None,Solano,Metropolitan Transportation Commission
68,Stanislaus Council of Governments,90311,None,Stanislaus,Stanislaus Council of Governments


4/28/2025 - added rtpa to those agencies. 
4/28/2025 - changes took affect.


## What are the unique RTPA in dim_orgs
- are any of the SOCAL CTCs in there?


There are some County Transportation Commission in the RTPA list
- Ventura County Transportation Commission

Could not find
- Los Angeles County Metropolitan Transportation Authority
- San Bernardino Associated Governments
- Riverside County Transportation Commission
- Imperial County Transportation Commission
- Orange County Transportation Authority

In [17]:
just_rtpa_name = (
    dim_orgs[dim_orgs["rtpa_name"].notna()]["rtpa_name"]
    .drop_duplicates()
    .reset_index(drop=True)
)

just_rtpa_name.info()

<class 'pandas.core.series.Series'>
RangeIndex: 43 entries, 0 to 42
Series name: rtpa_name
Non-Null Count  Dtype 
--------------  ----- 
43 non-null     object
dtypes: object(1)
memory usage: 476.0+ bytes


In [18]:
check_ctc = [
    "Los Angeles County Metropolitan Transportation Authority",
    "San Bernardino Associated Governments",
    "Riverside County Transportation Commission",
    "Imperial County Transportation Commission",
    "Orange County Transportation Authority",
    "Ventura County Transportation Commission",
]
just_rtpa_name[just_rtpa_name.isin(check_ctc)]

22    Ventura County Transportation Commission
Name: rtpa_name, dtype: object

In [19]:
# fuzzy string search

ctc_substring = [
    "Los Angeles",
    "Bernardino ",
    "Riverside",
    "Imperial",
    "Orange",
    "Ventura",
]

for i in ctc_substring:
    print(just_rtpa_name[just_rtpa_name.str.contains(i)])

Series([], Name: rtpa_name, dtype: object)
Series([], Name: rtpa_name, dtype: object)
Series([], Name: rtpa_name, dtype: object)
Series([], Name: rtpa_name, dtype: object)
Series([], Name: rtpa_name, dtype: object)
22    Ventura County Transportation Commission
Name: rtpa_name, dtype: object


In [20]:
# check ctc list against agency names col
dim_org_county[dim_org_county["organization_name"].isin(check_ctc)]

,name,ntd_id_2022,rtpa_name,key,organization_key,organization_name,county_geography_name,county_geography_key
59,Riverside County Transportation Commission,90218,Southern California Association of Governments,cb867447a408d161387ac5e2eeaeb7fb,cb867447a408d161387ac5e2eeaeb7fb,Riverside County Transportation Commission,Riverside,17a6841cb057ea751e22785fb9229596
175,Imperial County Transportation Commission,90226,Southern California Association of Governments,621fbb855822846119c106fd5bfa56b3,621fbb855822846119c106fd5bfa56b3,Imperial County Transportation Commission,Imperial,56a0304a6b0ccdc0dd0dbfae798a45e2
183,Los Angeles County Metropolitan Transportation Authority,90154,Southern California Association of Governments,9e96bde610e80d71f500eea119c4723c,9e96bde610e80d71f500eea119c4723c,Los Angeles County Metropolitan Transportation Authority,Los Angeles,8a8da539caf4f046025b97a5b4b9564b
194,Orange County Transportation Authority,90036,Southern California Association of Governments,47552ed0c038e35ee6f21ec8eb2cb5d8,47552ed0c038e35ee6f21ec8eb2cb5d8,Orange County Transportation Authority,Orange,aef4652aba294c8fd5ee922f9741ff6a
232,Ventura County Transportation Commission,90164,Ventura County Transportation Commission,4f7fa398d9c1c8c75310e13df4818015,4f7fa398d9c1c8c75310e13df4818015,Ventura County Transportation Commission,Ventura,ac21d3cfb432219540f51c7658df90e9


In [21]:
# somehow San Bernardino was missing from the previous cell
dim_org_county[dim_org_county["name"].str.contains("San Bernardino")]

,name,ntd_id_2022,rtpa_name,key,organization_key,organization_name,county_geography_name,county_geography_key
205,San Bernardino County Transportation Authority,90302,Southern California Association of Governments,c3ecb38b2814c6d88c9fd86f8cd14e70,c3ecb38b2814c6d88c9fd86f8cd14e70,San Bernardino County Transportation Authority,San Bernardino,1a3bb95333ae89740a2bfa8f5ba83d0d


## Socal CTC dictionary
to be used in other notebooks to convert socal counties from SCAG to the county CTC


In [22]:
dim_org_county[
    (dim_org_county["name"].isin(check_ctc))
    | (dim_org_county["name"].str.contains("San Bernardino"))
][["county_geography_name", "name"]]

,county_geography_name,name
59,Riverside,Riverside County Transportation Commission
175,Imperial,Imperial County Transportation Commission
183,Los Angeles,Los Angeles County Metropolitan Transportation Authority
194,Orange,Orange County Transportation Authority
205,San Bernardino,San Bernardino County Transportation Authority
232,Ventura,Ventura County Transportation Commission


In [23]:
# save this to a module for later imports
scag_to_ctc = {
    "Riverside": "Riverside County Transportation Commission",
    "Imperial": "Imperial County Transportation Commission",
    "Los Angeles": "Los Angeles County Metropolitan Transportation Authority",
    "Orange": "Orange County Transportation Authority",
    "San Bernardino": "San Bernardino County Transportation Authority",
    "Ventura": "Ventura County Transportation Commission",
}

scag_to_ctc

{'Riverside': 'Riverside County Transportation Commission',
 'Imperial': 'Imperial County Transportation Commission',
 'Los Angeles': 'Los Angeles County Metropolitan Transportation Authority',
 'Orange': 'Orange County Transportation Authority',
 'San Bernardino': 'San Bernardino County Transportation Authority',
 'Ventura': 'Ventura County Transportation Commission'}

## Compare ntd ID from dim_org_county to ntd id in xwalk

In [24]:
xwalk_compare = xwalk.merge(
    dim_org_county,
    how="outer",
    left_on="ntd_id",
    right_on="ntd_id_2022",
    indicator=True,
)
xwalk_compare["_merge"].value_counts()

both          224
left_only      30
right_only     15
Name: _merge, dtype: int64

these agencies do no appear in dim_orgs

they only appear in the xwalk (ntd id were pulled from an ntd table)

In [25]:
# unique agencies in the xwalk
xwalk_compare[xwalk_compare["_merge"] == "left_only"]

left_only_ntd_id = (
    xwalk_compare[xwalk_compare["_merge"] == "left_only"]["ntd_id"].unique().tolist()
)
display(len(left_only_ntd_id), type(left_only_ntd_id))

30

list

In [26]:
# unique agencies in dim_orgs
xwalk_compare[xwalk_compare["_merge"] == "right_only"]

right_only_ntd_id = (
    xwalk_compare[xwalk_compare["_merge"] == "right_only"]["ntd_id_2022"]
    .unique()
    .tolist()
)
display(len(right_only_ntd_id), type(right_only_ntd_id))

15

list

### checking if unique xwalk ntd_id appear anywhere in unfiltered dim_orgs

In [27]:
left_only_dim_orgs = (
    tbls.mart_transit_database.dim_organizations()
    >> filter(
        # _.ntd_id_2022.isin(left_only_ntd_id), # ntd_ids from xwalk, returned 0 matches
        _.ntd_id.isin(left_only_ntd_id)  # returned 0 matches
    )
    >> distinct(
        _.name,
        _.ntd_id,
        _.ntd_id_2022,
        _.rtpa_name,
        _.key,
        _._is_current,
        _.public_currently_operating,
    )
    >> collect()
)

len(left_only_dim_orgs)

# so the ntd_ids from the xwalk (initialy pulled from mart_ntd) DO NOT APPEAR anywhere in dim_orgs
# what can we conclude.

0

In [28]:
right_only_dim_orgs = (
    tbls.mart_transit_database.dim_organizations()
    >> filter(
        _.ntd_id_2022.isin(
            right_only_ntd_id
        ),  # ntd_id from dim_org. returned 31 matches
        # _.ntd_id.isin(right_only_ntd_id) # returned 43 matches
    )
    >> distinct(
        _.name,
        _.ntd_id,
        _.ntd_id_2022,
        _.rtpa_name,
        _.key,
        _._is_current,
        _.public_currently_operating,
    )
    >> collect()
)

len(right_only_dim_orgs)

31

In [38]:
### further investigate the unique xwalk ntd_id (these ids come from mart_ntd.)
# are there any similar ntd names in dim orgs?

xwalk_compare[xwalk_compare["_merge"] == "left_only"].sort_values(by="agency_name").reset_index(drop=True).value_counts("agency_name")

agency_name
ATC / Vancom                                                                                         1
Chico Area Transit System City of Chico (CATS)                                                       1
Ryder/ATE                                                                                            1
Paso Robles Transit Services (PE)                                                                    1
Paratransit, Inc. CTSA                                                                               1
Outreach & Escort dba OUTREACH                                                                       1
National City Transit (NCT)                                                                          1
Monterey County RIDES                                                                                1
MTS Contract Services (MCS)                                                                          1
Los Angeles County Transportation Commission / MTA           

A lot of these agency names includ acronyms in their names. or are sub-departments of the parent agency (public works).



In [30]:
check_names = [
    "ATC",
    "Vancom",
    "Chico",
    "Chula Vista",
    "Alameda Ferry",
    "Benicia",
    "Irvine",
    "Merced",
    "Vallejo",
    "San Diego",
    "DAVE",
    "Imperial",
    "LACMTA",
    "Laidwal",
    "Department of Public Works",
    "MTA",
    "MTS",
    "Monterey",
    "National City",
    "Outreach",
    "Paratransit",
    "Paso Robles",
    "Ryder",
    "San Diego Trolley",
    "San Gabriel",
    "Stanislaus",
]

In [31]:
all_dim_orgs = (
    tbls.mart_transit_database.dim_organizations()
    >> filter(
        # _._is_current == True,
        # _.name.LIKE("%San%")
    )
    # >> distinct(
    #   _.name,
    #  _.ntd_id,
    # _.ntd_id_2022,
    # _.rtpa_name,
    # _.key,
    # _._is_current,
    # _.public_currently_operating
    # )
    >> collect()
)
all_dim_orgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10026 entries, 0 to 10025
Data columns (total 28 columns):
 #   Column                                  Non-Null Count  Dtype              
---  ------                                  --------------  -----              
 0   key                                     10026 non-null  object             
 1   source_record_id                        10026 non-null  object             
 2   name                                    10018 non-null  object             
 3   organization_type                       8974 non-null   object             
 4   roles                                   10026 non-null  object             
 5   itp_id                                  5226 non-null   float64            
 6   details                                 1283 non-null   object             
 7   website                                 6309 non-null   object             
 8   reporting_category                      5200 non-null   object             


In [32]:
for i in check_names:
    display(
        f"""Results for agency_name containing: {i}""",
        all_dim_orgs[all_dim_orgs["name"].str.contains(i, na=False)]["name"]
        .unique()
        .tolist(),
        # all_dim_orgs[all_dim_orgs["name"].str.contains(i, na=False)]#.value_counts("name")
    )

'Results for agency_name containing: ATC'

[]

'Results for agency_name containing: Vancom'

[]

'Results for agency_name containing: Chico'

...

['San Diego Trolley, Inc.']

'Results for agency_name containing: San Gabriel'

['San Gabriel Transit Inc.', 'City of San Gabriel']

'Results for agency_name containing: Stanislaus'

['Historic: Stanislaus Regional Transit',
 'MOVE Stanislaus',
 'Stanislaus Council of Governments',
 'Stanislaus County',
 'Stanislaus Regional Transit',
 'Stanislaus Regional Transit Authority',
 'Stanislaus State']

## Copy pasted list of orgs with mart_ntd Id but no ntd_id in dim_org

| ntd_id | agency_name                                                       | city          | agency_status | Closest agency_name equivilent                           | Closest ntd_id equivilent | ntd_id_2022 updated in airtable                                                                                                 |   |   |   |   |   |   |   |   |   |   |
|--------|-------------------------------------------------------------------|---------------|---------------|----------------------------------------------------------|---------------------------|---------------------------------------------------------------------------------------------------------------------------------|---|---|---|---|---|---|---|---|---|---|
| 90174  | City of Benicia (Benicia Breeze)                                  | Benicia       | Inactive      | City of Benicia                                          | recHFNGB6YV8CuioA         | TRUE                                                                                                                            |   |   |   |   |   |   |   |   |   |   |
| 90231  | City of Irvine (COI)                                              | Irvine        | Inactive      | City of Irvine                                           | recK4si1uIoj6HfrO         | TRUE                                                                                                                            |   |   |   |   |   |   |   |   |   |   |
| 90021  | Los Angeles County Metropolitan Transportation Authority (LACMTA) | Los Angeles   | Inactive      | Los Angeles County Metropolitan Transportation Authority | 90154                     | ntd id already existed, no changes                                                                                              |   |   |   |   |   |   |   |   |   |   |
| 90224  | Paratransit, Inc. CTSA                                            | Sacramento    | Inactive      | Paratransit Inc.                                         | 90223                     | ntd id already existed, no changes                                                                                              |   |   |   |   |   |   |   |   |   |   |
| 90054  | San Diego Trolley, Inc. (MTS)                                     | San Diego     | Inactive      | San Diego Trolley, Inc.                                  | rec3k2j1TaChhTUg1         | TRUE                                                                                                                            |   |   |   |   |   |   |   |   |   |   |
| 90187  | San Gabriel Transit, Inc. (SGT)                                   | Rosemead      | Inactive      | San Gabriel Transit Inc.                                 | recEgsVErZ8aiiQoK         | TRUE                                                                                                                            |   |   |   |   |   |   |   |   |   |   |
| 90311  | Stanislaus Council of Governments (StanCOG) - Mobility Programs   | Modesto       | Active        | Stanislaus Council of Governments                        | recJQMJsqPJX18dpI         | TRUE                                                                                                                            |   |   |   |   |   |   |   |   |   |   |
| 90170  | ATC / Vancom                                                      | Oakland       | Inactive      | double none                                              |                           |                                                                                                                                 |   |   |   |   |   |   |   |   |   |   |
| 90193  | Chula Vista Transit (CVT)                                         | Chula Vista   | Inactive      | absorbed to MTS ?                                        | City of Chula Vista?      |                                                                                                                                 |   |   |   |   |   |   |   |   |   |   |
| 90212  | Imperial Valley Transit (IVT)                                     | El Centro     | Inactive      | Imperial County Transportation Commission                | rec38PbjPbEy2Tvdu         | IVT is listed as a service under ICTC, but IVT has its own website https://www.ivtransit.com/                                   |   |   |   |   |   |   |   |   |   |   |
| 90150  | City of Alameda Ferry Services                                    | Alameda       | Inactive      | City of Alameda                                          | reczluQLW1y5oQqF8         | TRUE                                                                                                                            |   |   |   |   |   |   |   |   |   |   |
| 90143  | City of Merced Transit System                                     | Merced        | Inactive      | City of Merced                                           | recL5F51uYuLlZFMh         | TRUE                                                                                                                            |   |   |   |   |   |   |   |   |   |   |
| 90189  | National City Transit (NCT)                                       | National City | Inactive      | City of National City                                    | rec1S4pn6HGRfSekT         | probably is City of Natnioal City. National city wiki said It was absored into MTS                                              |   |   |   |   |   |   |   |   |   |   |
| 90195  | Paso Robles Transit Services (PE)                                 | Paso Robles   | Inactive      | City of Paso Robles                                      | recTsBhbc04OTbbe3         | probably is City of Paso Robles, there is a "Paso Express" picture on the SLORTA website for Paso Robles Routes A & B services  |   |   |   |   |   |   |   |   |   |   |
| 90158  | DAVE Transportation Services, Inc.                                | Sherman Oaks  | Inactive      | double none                                              |                           |                                                                                                                                 |   |   |   |   |   |   |   |   |   |   |
| 90028  | City of Vallejo Transportation Program (Vallejo Transit, Baylink) | Vallejo       | Inactive      | City of Vallejo                                          | recUwbn2HNnI2rpxY         | SolTrans services Vallejo now, need to talk this one out                                                                        |   |   |   |   |   |   |   |   |   |   |
| 90166  | LACMTA - Small Operators (LACMTA)                                 | Los Angeles   | Inactive      | Los Angeles County Metropolitan Transportation Authority | 90154                     | put as alias on 90154?                                                                                                          |   |   |   |   |   |   |   |   |   |   |
